# Tp3 : NLP Classification de Texte

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Charger le dataset
df = pd.read_csv("movie_review.csv")
print(df.head())

   fold_id cv_tag  html_id  sent_id  \
0        0  cv000    29590        0   
1        0  cv000    29590        1   
2        0  cv000    29590        2   
3        0  cv000    29590        3   
4        0  cv000    29590        4   

                                                text  tag  
0  films adapted from comic books have had plenty...  pos  
1  for starters , it was created by alan moore ( ...  pos  
2  to say moore and campbell thoroughly researche...  pos  
3  the book ( or " graphic novel , " if you will ...  pos  
4  in other words , don't dismiss this film becau...  pos  


In [ ]:
#Pré-traitement (Pre-processing )des données textuelles
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convertir en minuscules
    text = text.lower()
    # Supprimer les stop words
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

df['processed_text'] = df['text'].apply(preprocess_text)

print(df['processed_text'])

0        films adapted comic books plenty success wheth...
1        starters created alan moore eddie campbell bro...
2        say moore campbell thoroughly researched subje...
3        book graphic novel 500 pages long includes nea...
4                           words dont dismiss film source
                               ...                        
64715      lack inspiration traced back insipid characters
64716    like many skits current incarnation saturdayni...
64717    watching one roxbury skits snl come away chara...
64718                        bump unsuspecting women thats
64719       watching anightattheroxbury youll left exactly
Name: processed_text, Length: 64720, dtype: object


In [ ]:
#Entraînement du modèle Word2Vec
tokenized_text = [word_tokenize(text) for text in df['processed_text']]
#La tokenization divise chaque texte de la colonne processed_text en une liste de mots (ou tokens).
word2vec_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)
# Afficher le vecteur pour un mot spécifique (par exemple, 'movie')
word_vector = word2vec_model.wv['movie']
print("Vecteur pour le mot 'movie':", word_vector)


Vecteur pour le mot 'movie': [-0.7021366   0.568045    0.7448934   0.67349255  0.18114206 -1.0586964
  2.0526474   1.5601648  -2.1281545  -0.43298176 -0.5941331  -1.2646284
  0.77901334  0.779618    0.04520872 -0.9423847   0.8527799  -0.49041417
 -0.7018608  -1.9959956   0.6741901  -0.22534531  0.62059826 -1.323488
 -0.44091684  0.16065687 -0.97726685 -0.06948443 -1.2435945  -0.01808226
  0.8546094  -0.08810988  0.90132314 -0.20489883 -0.39073825  1.1316277
  0.6912175   0.20495531 -0.14154923 -1.8532808   0.19506171 -0.27355957
 -0.71059585  0.63829     0.44012824 -0.18142009 -0.5368987  -0.29997614
  0.612479    0.47255173 -0.2966137  -0.15265651 -0.81501895  0.01716703
 -0.0318241  -0.22544399  0.8215619  -0.6543784  -0.8585152   0.92699295
  0.00499394 -0.36601704  0.06998275 -0.5575904  -1.5195374   0.94812256
  0.50012547  0.6007947  -1.6962597   1.2874672  -1.2753365   0.30237907
  1.0327144  -0.22284757  0.47078875 -0.5712742   0.5600681   0.36441335
 -0.6375482   0.25943977 -0

In [ ]:
# Vectorisation des reviews de movies
def vectorize_text(text, model):
    words = word_tokenize(text)
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return [0] * model.vector_size

df['vectorized_text'] = df['processed_text'].apply(lambda x: vectorize_text(x, word2vec_model))
print(df['vectorized_text'].head())

0    [-0.24738699, 0.44466636, 0.22824238, 0.09249,...
1    [-0.184112, 0.29823, 0.11517828, 0.11264722, 0...
2    [-0.2228232, 0.45173407, 0.23681158, 0.0067303...
3    [-0.1954853, 0.2628056, 0.1539347, 0.1190092, ...
4    [-0.2531006, 0.36936074, 0.3380906, 0.18004978...
Name: vectorized_text, dtype: object


In [ ]:
# Division des données
X = pd.DataFrame(df['vectorized_text'].to_list())
y = df['tag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#print("Ensemble d'entraînement X_train :\n", X_train.head())
#print("\nEnsemble de test X_test :\n", X_test.head())
#print("\nEnsemble d'entraînement y_train :\n", y_train.head())
#print("\nEnsemble de test y_test :\n", y_test.head())



In [ ]:
# Construction d'un classificateur
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
# Évaluation du modèle
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Affichage des résultats
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.5682941903584673
Precision: 0.5698996581019949
Recall: 0.5682941903584673
F1 score: 0.5629087775036085
